In [16]:
import os
import copy
import time
import random
from tqdm import tqdm

import PIL
from PIL import Image


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torchvision import datasets, transforms, models

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score
# fix seeds
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 2019
seed_everything(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.enabled = False
print(device)

cuda


### Data
- 196종의 차

In [17]:
df_class = pd.read_csv('./data/class.csv')
df_class.head()
# print(df_class.shape)

,id,name
0,1,AM General Hummer SUV 2000
1,2,Acura RL Sedan 2012
2,3,Acura TL Sedan 2012
3,4,Acura TL Type-S 2008
4,5,Acura TSX Sedan 2012


In [18]:
df_train = pd.read_csv('./data/train.csv')
df_train = df_train[['img_file', 'class']]
df_train.replace(196, 0, inplace=True)
df_train.head()

,img_file,class
0,train_00001.jpg,108
1,train_00002.jpg,71
2,train_00003.jpg,76
3,train_00004.jpg,188
4,train_00005.jpg,44


In [19]:
df_test = pd.read_csv('./data/test.csv')
df_test.head()

,img_file,bbox_x1,bbox_y1,bbox_x2,bbox_y2
0,test_00001.jpg,199,184,1116,809
1,test_00002.jpg,55,61,480,257
2,test_00003.jpg,42,123,602,413
3,test_00004.jpg,13,8,619,393
4,test_00005.jpg,8,9,209,93


### Train vs Validation Data split
- 신경망의 validation 을 위해 dataset을 분리합니다.

In [20]:
X_train, X_val, y_train, y_val = train_test_split(df_train['img_file'], df_train['class'], stratify=df_train['class'], test_size=0.2, random_state=SEED)
print(X_train.head(), y_train.head())
print(X_val.head(), y_val.head())

X_train = X_train.values
X_val = X_val.values
y_train = y_train.values
y_val = y_val.values

566     train_00567.jpg
592     train_00593.jpg
3074    train_03075.jpg
210     train_00211.jpg
7252    train_07253.jpg
Name: img_file, dtype: object 566     118
592     117
3074    172
210      79
7252    154
Name: class, dtype: int64
9036    train_09037.jpg
6581    train_06582.jpg
6541    train_06542.jpg
5061    train_05062.jpg
3056    train_03057.jpg
Name: img_file, dtype: object 9036    151
6581     81
6541    155
5061     76
3056    114
Name: class, dtype: int64


In [21]:
X_train

array(['train_00567.jpg', 'train_00593.jpg', 'train_03075.jpg', ...,
       'train_02049.jpg', 'train_01605.jpg', 'train_04788.jpg'],
      dtype=object)

In [7]:
print(X_train.shape)
print(X_val.shape)

(7992,)
(1998,)


### Data Preparation

#### Torch Dataset

In [8]:
TRAIN_DATA_PATH = './data/train_crop/'
TEST_DATA_PATH = './data/test_crop/'

class TrainImages(Dataset):
    def __init__(self, images, labels, mode=None, transforms=None):
        self.images = images
        self.labels = labels
        self.mode = mode
        self.transforms = transforms[self.mode]
        
    def __len__(self):
        return self.images.shape[0]
        
    def __getitem__(self, idx):
        image = Image.open(TRAIN_DATA_PATH + self.images[idx]).convert("RGB")
        image = self.transforms(image)
        label = self.labels[idx]
        
        return image, label
    
    
class TestImages(Dataset):
    def __init__(self, images, labels, mode=None, transforms=None):
        self.images = images
        self.laels = labels
        self.mode = mode
        self.transforms = transforms[self.mode]
        
    def __getitem__(self, idx):
        image = Image.open(TEST_DATA_PATH + self.images[idx]).convert("RGB")
        image = self.transforms(image)
        labels = self.labels[idx]
        
        return image, label

#### Image Transforms

In [22]:
transform = {
    'train': transforms.Compose([
        transforms.Resize((250, 250)),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406], 
            [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((250, 250)),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406], 
            [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((250, 250)),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406], 
            [0.229, 0.224, 0.225])
    ])
}

In [10]:
batch_size = 256

train_dataset = TrainImages(images=X_train, labels=y_train, mode='train', transforms=transform)
val_dataset = TrainImages(images=X_val, labels=y_val, mode='val', transforms=transform)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader
}

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset)
}

### Modeling
- resnext

In [11]:
model_res = models.resnet18(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)

#### Trainining

In [12]:
optimizer = optim.Adam(model_res.parameters())
criterion = nn.CrossEntropyLoss()

In [13]:
def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, device, epochs=20):
    start = time.time()
    
    num_classes = 196
    
    best_model_weights = copy.deepcopy(model.state_dict())
    best_f1 = 0.0
    
    for epoch in tqdm(range(epochs)):
        print("EPOCH {} / {}: ".format(epoch+1, epochs))
        print("-" * 10)
        
        epoch_loss = 0.0
        phase = 'train'
        
        for batch_index, (batch_inputs, batch_labels) in enumerate(dataloaders[phase]):
            batch_inputs = batch_inputs.cuda()
            batch_labels = batch_labels.cuda()
            
            optimizer.zero_grad()
            outputs = model(batch_inputs)
#             _, preds = torch.max(outputs, 1)
            batch_loss = criterion(outputs, batch_labels)
            batch_loss.backward()
            optimizer.step()
            epoch_loss += batch_loss.item() * batch_inputs.size(0)
            if batch_index % 5 == 0:
                print("EPOCH {} BATCH {}: training batch loss: {}".format(epoch+1, batch_index+1, batch_loss.item()))
            
            if batch_index % 10 == 0:
                phase = 'val'
                val_preds = np.zeros((dataset_sizes['val'], 1))
                val_loss = 0.0
                with torch.no_grad():
                    model.eval()
                    for val_batch_index, (val_batch_inputs, val_batch_labels) in enumerate(dataloaders[phase]):
                        
                        val_batch_inputs = val_batch_inputs.cuda()
                        val_batch_labels = val_batch_labels.cuda()
                        
                        val_outputs = model(val_batch_inputs).detach()
                        _, val_batch_preds = torch.max(val_outputs, 1)
                        val_batch_loss = criterion(val_outputs, val_batch_labels)
                        val_preds[val_batch_index * batch_size: (val_batch_index+1) * batch_size] = val_batch_preds.cpu().view(-1, 1).numpy()
                        val_loss += val_batch_loss.item() * val_batch_inputs.size(0)
                            
                    val_score = f1_score(y_val, val_preds, average='micro')
                    print()
                    print(">>>>>>  EPOCH {} BATCH {}: validation score {}".format(epoch+1, batch_index+1, val_score))
                    print()
                    if val_score > best_f1:
                        best_f1 = val_score
                        best_model_weights = copy.deepcopy(model.state_dict())
#                         torch.save(model.state_dict(), './model/first_trial/fine_tune/best_model_{}_{}.pt'.format(epoch, batch_index))
                        
                phase = 'train'
                model.train()
                
        epoch_loss = epoch_loss / dataset_sizes['train']
        print("EPOCH {}: EPOCH_LOSS: {}".format(epoch+1, epoch_loss))
        
    end = time.time()
    elapsed_time = end - start
    print("Training COMPLETED: {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("BEST VALIDATION F1: {:4f}".format(best_f1))
    
    model.load_state_dict(best_model_weights)
    return model

In [13]:
model_res.to(device)
train_model(model_res, dataloaders, dataset_sizes, criterion, optimizer, device, epochs=20)

  0%|          | 0/20 [00:00<?, ?it/s]

EPOCH 1 / 20: 
----------
EPOCH 1 BATCH 1: training batch loss: 5.556825637817383

>>>>>>  EPOCH 1 BATCH 1: validation score 0.011011011011011011

EPOCH 1 BATCH 6: training batch loss: 5.04494047164917
EPOCH 1 BATCH 11: training batch loss: 4.401520729064941

>>>>>>  EPOCH 1 BATCH 11: validation score 0.01701701701701702

EPOCH 1 BATCH 16: training batch loss: 3.9667809009552
EPOCH 1 BATCH 21: training batch loss: 3.5828843116760254

>>>>>>  EPOCH 1 BATCH 21: validation score 0.15515515515515516

EPOCH 1 BATCH 26: training batch loss: 3.094449043273926
EPOCH 1 BATCH 31: training batch loss: 2.7181689739227295

>>>>>>  EPOCH 1 BATCH 31: validation score 0.32682682682682684



  5%|▌         | 1/20 [03:35<1:08:16, 215.59s/it]

EPOCH 1: EPOCH_LOSS: 3.9904703535475172
EPOCH 2 / 20: 
----------
EPOCH 2 BATCH 1: training batch loss: 2.252932548522949

>>>>>>  EPOCH 2 BATCH 1: validation score 0.3148148148148148

EPOCH 2 BATCH 6: training batch loss: 1.873906135559082
EPOCH 2 BATCH 11: training batch loss: 1.6232776641845703

>>>>>>  EPOCH 2 BATCH 11: validation score 0.4744744744744745

EPOCH 2 BATCH 16: training batch loss: 1.4923676252365112
EPOCH 2 BATCH 21: training batch loss: 1.1947169303894043

>>>>>>  EPOCH 2 BATCH 21: validation score 0.5635635635635635

EPOCH 2 BATCH 26: training batch loss: 1.2016472816467285
EPOCH 2 BATCH 31: training batch loss: 0.9848882555961609

>>>>>>  EPOCH 2 BATCH 31: validation score 0.6316316316316316



 10%|█         | 2/20 [07:14<1:05:00, 216.71s/it]

...

 65%|██████▌   | 13/20 [47:24<25:34, 219.27s/it]

EPOCH 13: EPOCH_LOSS: 0.001694159942806826
EPOCH 14 / 20: 
----------
EPOCH 14 BATCH 1: training batch loss: 0.0016076378524303436

>>>>>>  EPOCH 14 BATCH 1: validation score 0.875875875875876

EPOCH 14 BATCH 6: training batch loss: 0.0014940351247787476
EPOCH 14 BATCH 11: training batch loss: 0.001475319266319275

>>>>>>  EPOCH 14 BATCH 11: validation score 0.8768768768768769

EPOCH 14 BATCH 16: training batch loss: 0.0015341416001319885
EPOCH 14 BATCH 21: training batch loss: 0.0014026351273059845

>>>>>>  EPOCH 14 BATCH 21: validation score 0.8773773773773774

EPOCH 14 BATCH 26: training batch loss: 0.0016283243894577026
EPOCH 14 BATCH 31: training batch loss: 0.0014258809387683868

>>>>>>  EPOCH 14 BATCH 31: validation score 0.8763763763763763



 70%|███████   | 14/20 [51:02<21:54, 219.06s/it]

EPOCH 14: EPOCH_LOSS: 0.0014880233347785096
EPOCH 15 / 20: 
----------
EPOCH 15 BATCH 1: training batch loss: 0.001422695815563202

>>>>>>  EPOCH 15 BATCH 1: validation score 0.875875875875876

EPOCH 15 BATCH 6: training batch loss: 0.001368064433336258
EPOCH 15 BATCH 11: training batch loss: 0.0013185366988182068


KeyboardInterrupt: 

## Fine Tuning

In [21]:
model_res = models.resnet18(pretrained=True, progress=False)
num_features = model_res.fc.in_features
model_res.fc = nn.Linear(num_features, 196)
model_res.load_state_dict(torch.load('./model/best_model_9_30.pt'))
model_res.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [22]:
optimizer = optim.Adam(model_res.parameters(), lr=0.0000001)
criterion = nn.CrossEntropyLoss()
train_model(model_res, dataloaders, dataset_sizes, criterion, optimizer, device, epochs=100)




  0%|          | 0/100 [00:00<?, ?it/s]

EPOCH 1 / 100: 
----------
EPOCH 1 BATCH 1: training batch loss: 0.0024226494133472443

>>>>>>  EPOCH 1 BATCH 1: validation score 0.8768768768768769

EPOCH 1 BATCH 6: training batch loss: 0.0022922679781913757
EPOCH 1 BATCH 11: training batch loss: 0.0025090686976909637

>>>>>>  EPOCH 1 BATCH 11: validation score 0.875875875875876

EPOCH 1 BATCH 16: training batch loss: 0.002293657511472702
EPOCH 1 BATCH 21: training batch loss: 0.002331729978322983

>>>>>>  EPOCH 1 BATCH 21: validation score 0.8768768768768769

EPOCH 1 BATCH 26: training batch loss: 0.002421341836452484
EPOCH 1 BATCH 31: training batch loss: 0.002515047788619995

>>>>>>  EPOCH 1 BATCH 31: validation score 0.8768768768768769






  1%|          | 1/100 [03:39<6:01:32, 219.12s/it]

EPOCH 1: EPOCH_LOSS: 0.0023080417231467363
EPOCH 2 / 100: 
----------
EPOCH 2 BATCH 1: training batch loss: 0.0022991150617599487

>>>>>>  EPOCH 2 BATCH 1: validation score 0.8763763763763763

EPOCH 2 BATCH 6: training batch loss: 0.0022609271109104156
EPOCH 2 BATCH 11: training batch loss: 0.0021486952900886536

>>>>>>  EPOCH 2 BATCH 11: validation score 0.8778778778778779

EPOCH 2 BATCH 16: training batch loss: 0.0022125840187072754
EPOCH 2 BATCH 21: training batch loss: 0.002396300435066223

>>>>>>  EPOCH 2 BATCH 21: validation score 0.8768768768768769

EPOCH 2 BATCH 26: training batch loss: 0.0021733902394771576
EPOCH 2 BATCH 31: training batch loss: 0.002283640205860138

>>>>>>  EPOCH 2 BATCH 31: validation score 0.8778778778778779






  2%|▏         | 2/100 [07:19<5:58:44, 219.64s/it]

...




 15%|█▌        | 15/100 [55:03<5:12:14, 220.40s/it]

EPOCH 15: EPOCH_LOSS: 0.002290520105730962
EPOCH 16 / 100: 
----------
EPOCH 16 BATCH 1: training batch loss: 0.0024439357221126556

>>>>>>  EPOCH 16 BATCH 1: validation score 0.8773773773773774

EPOCH 16 BATCH 6: training batch loss: 0.0022276416420936584
EPOCH 16 BATCH 11: training batch loss: 0.002574838697910309

>>>>>>  EPOCH 16 BATCH 11: validation score 0.8783783783783784

EPOCH 16 BATCH 16: training batch loss: 0.0022207386791706085
EPOCH 16 BATCH 21: training batch loss: 0.0021018274128437042

>>>>>>  EPOCH 16 BATCH 21: validation score 0.8778778778778779

EPOCH 16 BATCH 26: training batch loss: 0.0023655779659748077
EPOCH 16 BATCH 31: training batch loss: 0.002298180013895035

>>>>>>  EPOCH 16 BATCH 31: validation score 0.8763763763763763






 16%|█▌        | 16/100 [58:42<5:08:04, 220.06s/it]

EPOCH 16: EPOCH_LOSS: 0.002284391745421532
EPOCH 17 / 100: 
----------
EPOCH 17 BATCH 1: training batch loss: 0.0022891834378242493

>>>>>>  EPOCH 17 BATCH 1: validation score 0.8773773773773774

EPOCH 17 BATCH 6: training batch loss: 0.002269543707370758
EPOCH 17 BATCH 11: training batch loss: 0.002225469797849655

>>>>>>  EPOCH 17 BATCH 11: validation score 0.8773773773773774



KeyboardInterrupt: 